# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import h5py

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v1', file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v1
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


## Saving folders

In [ ]:
models_dir_ppo = 'models/PPO'
models_dir_td3 = 'models/TD3'

logdir = 'logs'

if not os.path.exists(models_dir_ppo):
    os.makedirs(models_dir_ppo)
    
if not os.path.exists(models_dir_td3):
    os.makedirs(models_dir_td3)

if not os.path.exists(logdir):
    os.makedirs(logdir)

## Twin Delayed DDPG (TD3) from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.2 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise=action_noise, batch_size=64, gamma=1.0, verbose=1, seed=0
            , tensorboard_log=logdir )

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=300, log_interval = 5, n_eval_episodes = 1)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## PPO with GAE from `stable_baseline3` 

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

total_t_steps = 20

model = PPO("MlpPolicy", env, verbose=1, gamma = 1.0, tensorboard_log=logdir, batch_size=2, n_steps=2)

# classstable_baselines3.ppo.PPO(policy, env, learning_rate=0.0003, n_steps=2048, 
#         batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
#         clip_range_vf=None, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, 
#         use_sde=False, sde_sample_freq=- 1, target_kl=None, tensorboard_log=None, 
#         create_eval_env=False, policy_kwargs=None, verbose=0, seed=None, device='auto', 
#         _init_setup_model=True)

for i in range(1,10):
    model.learn(total_timesteps=total_t_steps, reset_num_timesteps=False)

# learn(total_timesteps, callback=None, log_interval=1, eval_env=None, eval_freq=- 1, 
#       n_eval_episodes=5, tb_log_name='PPO', eval_log_path=None, reset_num_timesteps=True)

    model.save(f"{models_dir_ppo}/{total_t_steps*i}")

## From my `ddpg_agent.py` code

In [3]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [4]:
## Save all rewards, energies and princip dims in files during training
def create_info_h5(agent, env):
    # Check if file exist and creat it
    i = 0
    while os.path.exists(f'run_{i}.hdf5'):
        i += 1
    dataFile = h5py.File(f'run_{i}.hdf5', 'a')
    
    # Create dataset to store info in hdf5 file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id}
    st = h5py.string_dtype(encoding='utf-8')
    dataFile.create_dataset('info', dtype=st)
    for k in info.keys():
        dataFile['info'].attrs[k] = info[k]

    # Create dataset to store hyperparams of the model in hdf5 file
    hyperparams = {'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
                   , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
                  , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
                   , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
                  , 'add_noise_every':agent.add_noise_every}
    dataFile.create_dataset('hyperparams', dtype='f')
    for k in hyperparams.keys():
        dataFile['hyperparams'].attrs[k] = hyperparams[k]
    
    # Create group for rewards, energies, princip dims, actor and critic model
    dataFile.create_group('sigmas')
    dataFile.create_group('rewards')
    dataFile.create_group('energies')
    dataFile.create_group('princip_dims')
    dataFile.create_group('actor_models')
    dataFile.create_group('critic_models')
    
    return dataFile

def save_all(dat_file, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    # Create datasets for rewards, energies, pri dim and store data in it 
    dat_file['sigmas'].create_dataset(f'sigmas_ep_{i_ep}', dtype='f', data=sigmas_i_ep)
    dat_file['rewards'].create_dataset(f'rew_ep_{i_ep}', dtype='f', data=rew_i_ep)
    dat_file['energies'].create_dataset(f'en_ep_{i_ep}', dtype='f', data=en_i_ep)
    dat_file['princip_dims'].create_dataset(f'pri_dim_ep_{i_ep}', dtype='i', data=pri_dim_i_ep)
    
    # Store in actor models group the network params at each ep
    actor_model = torch.load(act_model_i_ep)
    dat_file['actor_models'].create_dataset(f'act_mod_{i_ep}', dtype='f')
    for k in actor_model.keys():
        dat_file['actor_models'][f'act_mod_{i_ep}'].attrs.create(name=k,data=actor_model[k].numpy())
    
    # Store in actor models group the network params at each ep
    critic_model = torch.load(cr_model_i_ep)
    dat_file['critic_models'].create_dataset(f'cri_mod_{i_ep}', dtype='f')
    for k in critic_model.keys():
        dat_file['critic_models'][f'cri_mod_{i_ep}'].attrs.create(name=k,data=critic_model[k].numpy())

def close_file(dat_file, actor_model_file, critic_model_file, file_sigmas):
    dat_file.close()
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 250, n_episodes=400):
    
    # Create h5 file and store info about alg and its hypereparams
    dat_file = create_info_h5(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(dat_file=dat_file, i_ep=int(i_ep), sigmas_i_ep=env.actions_taken \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , act_model_i_ep='checkpoint_actor.pth', cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    close_file(dat_file, 'checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return dat_file

In [6]:
all_data = run_ddpg()

#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [42.579    66.681305 52.615036]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0260603
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.272755681178598
#### CALL STEP #### 2
Action chosen at step:  [53.651127 78.20799  54.52775 ]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0238016
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.1156868947404615
#### CALL STEP #### 3
Action chosen at step:  [24.981234 79.55552  61.296425]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.00786723
With this action the full dim is:  3  and princip dim is:  3
##

With this action the energy is:  0.00983429
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1444076493757667
#### CALL STEP #### 6
Action chosen at step:  [41.12304  81.86181  53.510063]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  0.00842465
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.0463820399036585
#### CALL STEP #### 7
Action chosen at step:  [43.17302  99.000626 62.744633]
Basis size (it should be the same of full dim) =   7
With this action the energy is:  0.00787879
With this action the full dim is:  7  and princip dim is:  7
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.0084232283017975
#### CALL STEP #### 8
Action chosen at step:  [60.453247 85.89267  59.07898 ]
Basis size (it should be the same of full dim) =   8
With this action

With this action the energy is:  -0.0237358
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.19003840229953
#### CALL STEP #### 11
Action chosen at step:  [54.8681   60.965595 57.04852 ]
Basis size (it should be the same of full dim) =   11
With this action the energy is:  -0.0238654
With this action the full dim is:  11  and princip dim is:  11
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.1990507167823665
#### CALL STEP #### 12
Action chosen at step:  [47.044212 55.180637 72.04496 ]
Basis size (it should be the same of full dim) =   12
With this action the energy is:  -0.0250695
With this action the full dim is:  12  and princip dim is:  12
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  1.2827831849056928
#### CALL STEP #### 13
Action chosen at step:  [45.8684   31.495655 53.002426]
Basis size (it should be the same of full dim) =   13
With this

With this action the energy is:  -0.0361932
With this action the full dim is:  15  and princip dim is:  15
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.0563193161767312
#### CALL STEP #### 16
Action chosen at step:  [77.22835  12.898506 45.089504]
Basis size (it should be the same of full dim) =   16
With this action the energy is:  -0.0421738
With this action the full dim is:  16  and princip dim is:  16
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.472207032195641
#### CALL STEP #### 17
Action chosen at step:  [73.70442  27.752337 53.93089 ]
Basis size (it should be the same of full dim) =   17
With this action the energy is:  -0.0429996
With this action the full dim is:  17  and princip dim is:  17
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.529632721238282
#### CALL STEP #### 18
Action chosen at step:  [83.68446  46.994843 69.2099  ]
Basis size (it should be the same of full dim) =   18
With this

With this action the energy is:  -0.0634005
With this action the full dim is:  20  and princip dim is:  20
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  3.9483003642390164
Episode 4 ... Score: 6.309
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [37.47937  52.248898 55.961544]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0267901
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.3235055817154358
#### CALL STEP #### 2
Action chosen at step:  [56.89164  48.676704 60.790775]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0255649
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.238305830910102
#### CALL STE

...

With this action the energy is:  0.00436445
With this action the full dim is:  16  and princip dim is:  16
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.7640379097862802
#### CALL STEP #### 17
Action chosen at step:  [110. 110. 110.]
Basis size (it should be the same of full dim) =   17
With this action the energy is:  0.00432987
With this action the full dim is:  17  and princip dim is:  16
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.7616332351349193
#### CALL STEP #### 18
Action chosen at step:  [110. 110. 110.]
Basis size (it should be the same of full dim) =   18
With this action the energy is:  0.00432813
With this action the full dim is:  18  and princip dim is:  16
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.7615122364682527
#### CALL STEP #### 19
Action chosen at step:  [110.      109.63043 110.     ]
Basis size (it should be the same of full dim) =   19
With this action the energy is:  0

With this action the energy is:  0.0120433
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.2980210198675124
#### CALL STEP #### 2
Action chosen at step:  [110.        86.397835 110.      ]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0105991
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1975921265333138
#### CALL STEP #### 3
Action chosen at step:  [110.       100.725204 108.619736]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.00722207
With this action the full dim is:  3  and princip dim is:  3
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.9627552717650047
#### CALL STEP #### 4
Action chosen at step:  [110.       100.105286  93.68014 ]
Basis size (it should be the same of full dim) =   4
With this

With this action the energy is:  0.00672073
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.9278923570827118
#### CALL STEP #### 7
Action chosen at step:  [110.      104.78581  81.47571]
Basis size (it should be the same of full dim) =   7
With this action the energy is:  0.0067099
With this action the full dim is:  7  and princip dim is:  7
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.9271392446919027
#### CALL STEP #### 8
Action chosen at step:  [110.        95.86099   89.776566]
Basis size (it should be the same of full dim) =   8
With this action the energy is:  0.0062939
With this action the full dim is:  8  and princip dim is:  8
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.8982108278334131
#### CALL STEP #### 9
Action chosen at step:  [110.       71.46603  98.06654]
Basis size (it should be the same of full dim) =   9
With this actio

With this action the energy is:  7.83647e-18
With this action the full dim is:  11  and princip dim is:  11
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.46053639786795664
#### CALL STEP #### 12
Action chosen at step:  [110.       94.53508 110.     ]
Basis size (it should be the same of full dim) =   12
With this action the energy is:  3.24214e-17
With this action the full dim is:  12  and princip dim is:  12
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.46053639786795664
#### CALL STEP #### 13
Action chosen at step:  [110.       93.51082 106.37888]
Basis size (it should be the same of full dim) =   13
With this action the energy is:  -1.25818e-19
With this action the full dim is:  13  and princip dim is:  13
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.46053639786795664
#### CALL STEP #### 14
Action chosen at step:  [110.      109.28391 101.50127]
Basis size (it should be the same of full dim) =   

With this action the energy is:  0.00500171
With this action the full dim is:  16  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.8083526283613764
#### CALL STEP #### 17
Action chosen at step:  [110. 110. 110.]
Basis size (it should be the same of full dim) =   17
With this action the energy is:  0.00500148
With this action the full dim is:  17  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.808336634284748
#### CALL STEP #### 18
Action chosen at step:  [ 95.190094 110.       110.      ]
Basis size (it should be the same of full dim) =   18
With this action the energy is:  0.00500092
With this action the full dim is:  18  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.8082976921851284
#### CALL STEP #### 19
Action chosen at step:  [106.37465   91.413864 110.      ]
Basis size (it should be the same of full dim) =   19
With this acti

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break